In [0]:
#para cada arquivo existente no diretorio /Volumes/jsf_catalog/ecommerce/jsf_vol_ecommerce, crie um df. Em seguida, crie uma delta table baseado neste DF. As tabelas não devem ter DF_ nas suas nomenclaturas, nem o prefixo olist_, nem a extensão do arquivo. As tabelas devem ser criadas em jsf_catalog.ecommerce

In [0]:
import os
from pyspark.sql import SparkSession

# Set directory path
directory_path = "/Volumes/jsf_catalog/ecommerce/jsf_vol_ecommerce"

# Create SparkSession
spark = SparkSession.builder.getOrCreate()

# Loop through the files in the directory
for file_name in os.listdir(directory_path):
    if file_name.startswith("olist_"):  # Check if the file starts with "olist_"
        # Create DataFrame with the file contents
        df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(os.path.join(directory_path, file_name))
        
        # Get table name by removing the prefix "olist_" and the file extension
        table_name = file_name.replace("olist_", "").split('.')[0]
        table_name = table_name.replace("_dataset","")+"_bronze"
    
        # Create a Delta table with the DataFrame data
        df.write.format("delta").mode("overwrite").saveAsTable(f"jsf_catalog.ecommerce.{table_name}")

In [0]:
%sql
-- Crie uma tabela que tenha o resultado do join entre order, order item, pagamentos e cliente chame a tabela de vendas_ecommerce, mantenha só os principais campos e traduza os nomes para portugues
create or replace table jsf_catalog.ecommerce.vendas_silver
as
    SELECT 
        o.order_id AS id_pedido,
        o.order_purchase_timestamp AS data_compra,
        o.order_approved_at AS data_aprovacao,
        o.order_delivered_carrier_date AS data_entrega_transportadora,
        o.order_delivered_customer_date AS data_entrega_cliente,
        o.order_estimated_delivery_date AS data_entrega_estimada,
        oi.order_item_id AS id_item_pedido,
        oi.product_id AS id_produto,
        oi.seller_id AS id_vendedor,
        oi.price AS preco,
        oi.freight_value AS valor_frete,
        p.payment_type AS tipo_pagamento,
        p.payment_installments AS parcelas,
        c.customer_id AS id_cliente,
        c.customer_unique_id AS id_cliente_unico,
        c.customer_zip_code_prefix AS cep_cliente,
        c.customer_city AS cidade_cliente,
        c.customer_state AS estado_cliente,
        rev.review_score AS nota_cliente

    FROM jsf_catalog.ecommerce.orders_bronze AS o
    JOIN jsf_catalog.ecommerce.order_items_bronze AS oi 
        ON o.order_id = oi.order_id
    JOIN jsf_catalog.ecommerce.order_payments_bronze AS p 
        ON o.order_id = p.order_id
    JOIN jsf_catalog.ecommerce.customers_bronze AS c 
        ON o.customer_id = c.customer_id
    JOIN jsf_catalog.ecommerce.order_reviews_bronze rev
        ON o.order_id = rev.order_id


num_affected_rows,num_inserted_rows
